We'll use sklearn.naive_bayes to train a spam classifier, most of the code is just loading our training data into a pandas DataFrame that we can process

In [8]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

def read_files(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)
            in_body = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if in_body:
                    lines.append(line)
                elif line == '\n':
                    in_body = True
            f.close()
            message = '\n'.join(lines)
            yield path, message
def data_frame_from_directory(path, classification):
    rows, index = [], []
    for filename, message in read_files(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)
    
    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})
data = data.append(data_frame_from_directory('./emails/spam', 'spam'))
data = data.append(data_frame_from_directory('./emails/ham', 'ham'))

In [11]:
#Our dataframe:
data.head()

,class,message
./emails/spam/00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
./emails/spam/00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
./emails/spam/00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
./emails/spam/00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
./emails/spam/00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


Use CountVectorizer to split up each message into its list of words, and throw that into a MultinomialNB classifier. Call fit() and we're got a trained spam filter.

In [39]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)
classifier = MultinomialNB()
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [44]:
examples = ["Free Viagra Now!!!", "Hi, Bob, How about a game of golf tomorrow?", "Gob opportunity"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham', 'ham'], 
      dtype='<U4')